In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def make_env():
    from diffrl.env import DifferentiablePhysicsEnv

    env = DifferentiablePhysicsEnv(observer_cfg={"TYPE": "ParticleObserver"})

    state = env.get_state()

    def loss_fn(idx, **kwargs):
        return kwargs['dist'].min(axis=0)[0].sum()

    env.set_loss_fn(loss_fn)
    env.set_initial_state_sampler(lambda env: state)

    return env

from diffrl.ppo.vec_envs import SubprocVectorEnv
env = SubprocVectorEnv([make_env])

[Taichi] mode=release
[Taichi] preparing sandbox at /tmp/taichi-fcf7efvo
[Taichi] version 0.7.26, llvm 10.0.0, commit e37bdb5e, linux, python 3.8.12
[I 01/20/22 06:00:45.671 54922] [shell.py:_shell_pop_print@35] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=cuda
[Taichi] mode=release
[Taichi] preparing sandbox at /tmp/taichi-4jriuf7q
[Taichi] version 0.7.26, llvm 10.0.0, commit e37bdb5e, linux, python 3.8.12
[Taichi] Starting on arch=cuda
{'width': (0.2, 0.2, 0.2), 'init_pos': (0.5, 0.1, 0.5), 'n_particles': 3000, 'color': 9868800}
Building primitive
action:
  dim: 6
  scale: (0.01, 0.01, 0.01, 0.01, 0.01, 0.01)
color: (0.8, 0.8, 0.8)
friction: 1.0
init_pos: (0.0, 0.1, 0.5)
init_rot: (1.0, 0.0, 0.0, 0.0)
lower_bound: (-1.0, -1.0, -1.0)
mass: 1.0
round: 0.01
shape: Box
size: (0.02, 0.15, 0.35)
upper_bound: (2.0, 2.0, 2.0)
variations: None
Building primitive
action:
  dim: 6
  scale: (0.01, 0.01, 0.01, 0.01, 0.01, 0.01)
color: (0.8, 0.8, 0.8)
frictio

In [5]:
import numpy as np

print(env.reset(requires_grad=True)[0].shape)
print(env.step([env.action_space[0].sample()])[0][0].shape)


action = np.zeros((128, 12))

rl_step = 0

reward_grad = []
env.reset(requires_grad=True)
env.zero_grad()

step = 0
for a in action:
    env.step([a])
    reward_grad.append(1)

    step += 1
    if step == 50:
        env.inform_obs_reward_grads(0, None, reward_grad)
        reward_grad = []
        env.reset(0, requires_grad=True)
        step = 0 

env.inform_obs_reward_grads(0, None, reward_grad)
reward_grad = []


print(env.get_action_grads(0))

(200, 3)
(200, 3)
[[array([-0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,
        0. ], dtype=float32), array([-0.49000013,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.49000013,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ], dtype=float32), array([-0.48000005,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.48000005,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ], dtype=float32), array([-0.46999994,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.46999994,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ], dtype=float32), array([-0.46000007,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.46000007,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ], dtype=float32), array([-0.45000002,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.450000